In [1]:
import numpy as np
import pandas as pd
import sklearn.preprocessing as prep
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression

In [2]:
raw = pd.read_csv('Для математики.csv', decimal=',', dtype = {
    'возраст': 'Int64',
    'стадия': 'Int64',
    'тип кости по Mich': 'Int64',
    }, converters = {
    'пол': str.strip,
    'первичный очаг': str.strip,
    'Метастазы': pd.notna,
    'Лечение': lambda x: 'Лечение' if pd.notna(x) and x !='конс' else x,
    'Химиотерапия': pd.notna,
    'тип препарата': lambda x: '' if pd.isna(x) else x.replace('+', '|').replace(',', '|').replace(' ','|')
})
raw.dtypes

ФИО                                object
пол                                object
возраст                             Int64
первичный очаг                     object
Метастазы                            bool
Лечение                            object
Химиотерапия                         bool
диагноз                            object
срок приема ОМА (мес.)              int64
тип препарата                      object
длительность заболевания (год)    float64
стадия                              Int64
тип кости по Mich                   Int64
dtype: object

In [3]:
df = pd.get_dummies(raw[['возраст', 'Метастазы', 'Химиотерапия', 'срок приема ОМА (мес.)', 'длительность заболевания (год)', 'стадия', 'тип кости по Mich']], columns = ['стадия', 'тип кости по Mich']).join([
    raw['пол'].str.get_dummies(),
    raw['первичный очаг'].str.get_dummies(),
    raw['Лечение'].str.get_dummies(),
    raw['тип препарата'].str.get_dummies()
   ])\
    .dropna()
df.columns

Index(['возраст', 'Метастазы', 'Химиотерапия', 'срок приема ОМА (мес.)',
       'длительность заболевания (год)', 'стадия_1', 'стадия_2', 'стадия_3',
       'тип кости по Mich_1', 'тип кости по Mich_2', 'тип кости по Mich_3',
       'тип кости по Mich_4', 'тип кости по Mich_5', 'ж', 'м', 'Лифома',
       'желудок', 'кишечник', 'копчик', 'легкое', 'матка', 'миеломная болезнь',
       'молочн железа', 'остеопороз', 'почка', 'простата', 'ректосигм соед',
       'Лечение', 'конс', 'БФ', 'деносумаб'],
      dtype='object')

In [4]:
X = df[set(df.columns).difference(['длительность заболевания (год)'])].applymap(int)
y = df['длительность заболевания (год)']

In [5]:
regr = LinearRegression().fit(prep.maxabs_scale(X), y)
sorted(zip(X.columns, regr.coef_), key = lambda x: abs(x[1]), reverse = True)

[('возраст', -12.150697203029978),
 ('стадия_1', -8.121983340490303),
 ('Лифома', -7.88584614092921),
 ('легкое', 6.2121027749018),
 ('тип кости по Mich_3', 6.1870357193609715),
 ('ректосигм соед', -4.3950979309579274),
 ('деносумаб', -4.161512977791536),
 ('стадия_2', -4.149857892217787),
 ('стадия_3', -4.032233998072036),
 ('тип кости по Mich_2', 3.9351795863774672),
 ('тип кости по Mich_4', 3.549839132233468),
 ('кишечник', 3.4422491690233796),
 ('БФ', -3.229068734890281),
 ('копчик', 2.448053626635844),
 ('матка', -1.9872211429134121),
 ('м', -1.7592131315387798),
 ('ж', 1.7592131315387758),
 ('тип кости по Mich_1', -1.5172092831018076),
 ('Лечение', -1.4327438370463104),
 ('конс', 1.4327438370462984),
 ('тип кости по Mich_5', 1.2185552920913305),
 ('остеопороз', -1.2092440812681424),
 ('простата', 1.0085467999969628),
 ('почка', 0.9203075800673255),
 ('миеломная болезнь', 0.8577948633472857),
 ('срок приема ОМА (мес.)', -0.7374869258458057),
 ('молочн железа', 0.5883544820960931),

In [ ]:
raw.plot()